In [1]:
import numpy as np
import pandas as pd
import statistics
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error

In [49]:
#read file
df_data = pd.read_csv("data/dataset_mood_smartphone.csv")
df_data.head()

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [51]:
prev_day="2014-02-26"
new_data=pd.DataFrame(columns=['id', 'time', 'var',"value"])
temp_list=[]
for index, row in df_data.iterrows():
    date=row["time"].split(" ")[0]
    if date != prev_day:
        new_data.loc[index]=[df_data.loc[index-1]["id"], prev_day , df_data.loc[index-1]["variable"], statistics.mean(temp_list)]
        temp_list=[row["value"]]
    else:
        temp_list.append(row["value"])
    prev_day=date

In [52]:
new_data

,id,time,var,value
4,AS14.01,2014-02-26,mood,6.250000
7,AS14.01,2014-02-27,mood,6.333333
12,AS14.01,2014-03-21,mood,6.200000
17,AS14.01,2014-03-22,mood,6.400000
22,AS14.01,2014-03-23,mood,6.800000
27,AS14.01,2014-03-24,mood,6.000000
31,AS14.01,2014-03-25,mood,6.750000
36,AS14.01,2014-03-26,mood,6.600000
41,AS14.01,2014-03-27,mood,7.000000
46,AS14.01,2014-03-28,mood,6.400000


In [143]:
df_mood_id = new_data.loc[(new_data['var'] == "mood")]
df_time_value = df_mood_id[['id','time',"value"]].reset_index()
df_time_value['time'] = pd.to_datetime(df_time_value['time'],format="%Y-%m-%d")

X = []

id = 'AS14.01'
prev_index = 0
for i, row in df_time_value.iterrows():
    delta = (df_time_value['time'].iloc[prev_index] - df_time_value['time'].iloc[i]).days
    if delta < -1:
        for n in range(abs(delta)-1):
            X.append(None)
        X.append(df_time_value['value'].iloc[i])
    elif delta > 0:
        for n in range(365-delta-1):
            X.append(None)
        X.append(df_time_value['value'].iloc[i])
    else:
        X.append(df_time_value['value'].iloc[i])
    prev_index = i
            
print(history)
#Set up the input for ARIMA
# index = df_time_value[["time"]].values
# data_np = df_time_value["value"].values
series = pd.Series(X)
X= series.values
print(X)
np.savetxt("dataset_arima.txt", X)


[6.25, 6.333333333333333, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 6.2, 6.4, 6.8, 6.0, 6.75, 6.6, 7.0, 6.4, 8.0, 7.5, 7.4, 6.0, 6.5, 6.4, 6.2, 6.8, 6.5, 6.5, 6.6, 7.4, 7.4, 7.0, 7.25, 8.0, 7.2, 6.4, 7.2, 6.6, 6.8, 7.8, 7.25, 7.6, 7.4, 7.6, 7.6, 7.2, 7.6, 7.5, 7.6, 7.0, 7.8, 8.0, 7.6, 8.0, 8.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [140]:
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()
for t in range(len(test)):
    #order=(p,d,q) 
    model = ARIMA(history, order=(1,1,1))#tunne parameters!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

MissingDataError: exog contains inf or nans

In [15]:
df_time_value

,time,value
4,02-26,6.250000
7,02-27,6.333333
12,03-21,6.200000
17,03-22,6.400000
22,03-23,6.800000
27,03-24,6.000000
31,03-25,6.750000
36,03-26,6.600000
41,03-27,7.000000
46,03-28,6.400000
